In [1]:
from bs4 import BeautifulSoup
import csv
import pandas as pd
import random
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests
import time



def gettingCondition(condition):
    if (condition == '10') or (condition == '9.9') or (condition == '9.75') or (condition == '9.5'):
        mycondition = 'Like New'

    elif (condition == '9.25') or (condition == '9'):
        mycondition = 'Excellent'

    elif (condition == '8.75') or     (condition == '8.5') or (condition == '8'):
        mycondition = 'Good'

    elif (condition == '7.75') or (condition == '8.25'):
        mycondition = 'Fair'

    elif (condition) == '.':
        mycondition = 'New'
    else:
        pass
    return mycondition
    

def Ebay(main_Link,sheet):


    r = requests.get(main_Link)
    soup = BeautifulSoup(r.text, 'html.parser')
    Links = []

    for i in soup.findAll('h3',{'class':'lvtitle'}):
        try:
            Links.append(i.find('a').get('href'))
        except:
            Links.append('')
    print(len(Links))


    for link in Links:
        try:
            r = requests.get(link)
            time.sleep(random.randint(2,5))
            soup = BeautifulSoup(r.text, 'html.parser')


            values = soup.findAll('div',{'class':'ux-labels-values__values-content'})
            label_= soup.findAll('div',{'class':'ux-labels-values__labels-content'})

            model = '.'
            storage = '.'
            ram = '.'
            color = '.'
            HDC = '.'
            SSDC = '.'
            SZ = '.'
            PS = '.'
            MPN = '.'
            year = '.'
            Processor = '.'
            series = '.'
            product_family = '.'
            gpu = '.'
            seller_notes = '.'
            processor = '.'
            storage_type = '.'
            memory = '.'


            try:
                for i in range(len(label_)):
                    if label_[i].text.strip() == 'Model:':
                        model = (label_[i].findNext().findNext().findNext().text.strip())

                    if label_[i].text.strip() == 'Hard Drive Capacity:':
                        storage = (label_[i].findNext().findNext().findNext().text.replace('Apple SSD','').replace('SSD','').replace(' ','').strip())



                    if label_[i].text.strip() == 'GPU:':
                        gpu = (label_[i].findNext().findNext().findNext().text.strip())


                    if label_[i].text.strip() == 'SSD Capacity:':
                        SSDC = (label_[i].findNext().findNext().findNext().text.replace('Apple SSD','').replace('SSD','').replace(' ','').strip())

                    if label_[i].text.strip() == 'Screen Size:':
                        SZ = (label_[i].findNext().findNext().findNext().text.replace('in','').split('.')[0].replace(' ','').replace('"','').strip())+'"'


                    if label_[i].text.strip() == 'Product Family:':
                        product_family = (label_[i].findNext().findNext().findNext().text.strip())

                    if label_[i].text.strip() == 'Series:':
                        series = (label_[i].findNext().findNext().findNext().text.strip())


                    if label_[i].text.strip() == 'Processor Speed:':
                        PS = (label_[i].findNext().findNext().findNext().text.strip().split()[0])
                        a = PS
                        if len(a.split('GH')[0].strip().split('.')[-1]) > 1:
                            PS = a.split('GH')[0].strip().split('.')[0]+'.'+a.split('GH')[0].strip().split('.')[-1][0]+'GHz'
                        else:
                            PS = a.replace('GHz','')+'GHz'

                            
                    if label_[i].text.strip() == 'MPN:':
                        MPN = (label_[i].findNext().findNext().findNext().text.strip().split('LL/A')[0].split('LLA')[0]).replace('LL/A','').replace('LLA','').strip()

                    if label_[i].text.strip() == 'Release Year:':
                        year = (label_[i].findNext().findNext().findNext().text.strip().split()[-1].strip())


                    if label_[i].text.strip() == 'RAM Size:':
                        ram = (label_[i].findNext().findNext().findNext().text.replace('RAM','').replace(' ','').strip())

                    if label_[i].text.strip() == 'Memory:':
                        memory = (label_[i].findNext().findNext().findNext().text.replace('RAM','').replace(' ','').strip())

                    if label_[i].text.strip() == 'Storage Type:':
                        storage_type = (label_[i].findNext().findNext().findNext().text.replace('RAM','').split('(')[0].replace(' ','').strip())

                    if label_[i].text.strip() == 'Processor:':
                        processor = (label_[i].findNext().findNext().findNext().text.replace('RAM','').replace('Apple ','').replace('Intel ','').replace(' 8-core','').strip())
                        pro = ''
                        if processor != 'M1':
                            for ci in range(3,10):
                                if ci%2!=0:
                                    if 'Core i'+str(ci) in processor:
                                        pro = 'Core i'+str(ci)
                        if pro != '':
                            processor = pro

            except:
                pass



            try:
                my_condition = soup.find('span',{'class':'topItmCndDscMsg'}).text.split('/')[0].split('(')[-1]
                condition = gettingCondition(my_condition)
                
            except:
                condition = 'Sealed'

            try:
                price = soup.find('span',{'itemprop':'price'}).get('content').split('.')[0].strip()
            except:
                price = '.'


            try:
                title = soup.find('meta',{'property':'og:title'}).get('content').replace('eBay','').strip()
            except:
                title = '.'

            try:
                seller_notes = '.'
                for i in soup.findAll('div',{'class':'ux-labels-values__labels-content'}):
                    if i.text == 'Seller Notes:':
                        seller_notes = (i.findNext().findNext().findNext().text)
            except:
                pass

            try:
                variation = ' '
                applecare = '.'
                if 'vega' in title.lower():
                    variation = 'Vega'

                if 'applecare+' in title.lower():
                    applecare = 'AppleCare+'
            except:
                pass

            
            try:
                
                desp = series + ' ' + SZ + ' ' + processor + ' ' + PS +' ' + ram + ' ' + storage + ' ' + year + ' ' + variation
                desp = desp.replace('2667Mhz','').strip()
            except:
                desp = '.'

            data = [title,seller_notes,storage,SSDC,memory,MPN,PS,year,storage_type,gpu,processor,product_family,SZ,variation,applecare,desp,condition,price,link]
            sheet.append_row(data)
            print(data)
            
        except:
            pass
        


        
        

head = ['Title','Seller Notes','Harddrive Capacity','SSD Capacity','Memory','MPN','Processor Speed','Release Year','Storage Type','GPU','Processor','Product Family','Screen Size','Variation','Apple Care','Description','Condition','Price','URL']
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name("bussines_credits.json", scope)
client = gspread.authorize(creds)

spreadsheetName = 'Global Price'
sheetName = 'Apple Laptops-1'

spreadsheet = client.open(spreadsheetName)
sheet = spreadsheet.worksheet(sheetName)
sheet.clear()
sheet.append_row(head)


url = 'https://www.ebay.com/sch/Apple-Laptops/111422/m.html?_ssn=quality-macs&_from=R40&_nkw='
Ebay(url+'&_ipg=200&rt=nc',sheet)

records_data = sheet.get_all_records()
df = pd.DataFrame.from_dict(records_data)
df.to_csv('Cleaned Data.csv',index=False)

SyntaxError: invalid syntax (2/ipykernel_27128/1984899138.py, line 153)